In [ ]:
import os
from uuid import uuid4
from dotenv import load_dotenv

from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain.chains import RetrievalQA
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableSequence
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.schema import StrOutputParser
from langchain.tools import Tool
from langchain_community.chat_message_histories import Neo4jChatMessageHistory
from langchain_community.graphs import Neo4jGraph
from langchain_community.tools import YouTubeSearchTool
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import ChatOpenAI as Chat, OpenAIEmbeddings

load_dotenv()

In [2]:
openai_api_key = os.getenv("OPENAI_API_KEY")
model = os.getenv("OPENAI_MODEL", "gpt-4o")
temperature = float(os.getenv("OPENAI_TEMPERATURE", 0))

NEO4J_URL = os.getenv("NEO4J_URL")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

In [ ]:
llm = Chat(
    openai_api_key=openai_api_key,
    model=model,
    temperature=temperature
)

print(llm)

In [ ]:
embeddings = OpenAIEmbeddings()
print(embeddings)

In [ ]:
# Initialize Neo4j graph connection
graph = Neo4jGraph(
    url=NEO4J_URL,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD
)

print(graph)

In [ ]:
# Initialize Neo4j vector store connection
vector_store = Neo4jVector(
    embeddings,
    url=NEO4J_URL,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD
)

print(vector_store)

In [ ]:
# Generate a unique session ID
SESSION_ID = str(uuid4())
print(f"Session ID: {SESSION_ID}")


def get_memory(session_id):
    return Neo4jChatMessageHistory(session_id=session_id, graph=graph)

In [8]:
SCHEMA = """
(Movie)-[:IN_GENRE]->(Genre), (Movie)-[:DIRECTED_BY]->(Director), 
(Movie)-[:HAS_ACTOR]->(Actor), (Movie)-[:HAS_RATING]->(Rating)
"""

In [9]:
CYPHER_GENERATION_TEMPLATE = f"""
You are an expert Neo4j Developer translating user questions into Cypher to answer questions about movies and provide recommendations.
Convert the user's question based on the schema.

Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
For movie titles that begin with "The", move "the" to the end, For example "The 39 Steps" becomes "39 Steps, The" or "The Matrix" becomes "Matrix, The".

If no data is returned, do not attempt to answer the question.
Only respond to questions that require you to construct a Cypher statement.
Do not include any explanations or apologies in your responses.

Examples:
Find movies and genres:
MATCH (m:Movie)-[:IN_GENRE]->(g)
RETURN m.title, g.name

Schema: {SCHEMA}
Question: {{question}}
"""

few_shot_example = """
Find all movies directed by Christopher Nolan:
MATCH (m:Movie)-[:DIRECTED_BY]->(d:Director {name: 'Christopher Nolan'})
RETURN m.title
"""

formatted_query = CYPHER_GENERATION_TEMPLATE.format(question=few_shot_example)

In [ ]:
# Define the movie chat prompt template
movie_prompt = ChatPromptTemplate.from_messages([
    ("system", formatted_query),
    ("system", "{context}"),
    ("human", "{question}"),
    MessagesPlaceholder(variable_name="chat_history"),
])

print(movie_prompt)

In [ ]:
documents = graph.query("MATCH (n) RETURN n LIMIT 25;")

# Convert the retrieved documents into Document objects
document_objects = [Document(page_content=str(doc)) for doc in documents]

db = Neo4jVector.from_documents(
    document_objects, embeddings, url=NEO4J_URL, username=NEO4J_USERNAME, password=NEO4J_PASSWORD
)

print(db)

In [ ]:
retriever = RetrievalQAWithSourcesChain.from_llm(
    llm=llm,
    retriever=db.as_retriever(),
)

print(retriever)

# retriever = RetrievalQAWithSourcesChain.from_chain_type(
#     llm, chain_type="stuff", retriever=retriever
# )

In [ ]:
query = "hi!"
retriever.invoke(query)[0]

In [ ]:
rag_chain = (
    RunnablePassthrough.assign(context=(lambda x: self.format_docs(x["context"])))
    | movie_prompt
    | llm
    | StrOutputParser()
)

retriever.rag_chain_w_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain)

In [ ]:
# chat_chain = movie_prompt | llm | StrOutputParser()
# print(chat_chain)

In [25]:
youtube = YouTubeSearchTool()

def call_trailer_search(input):
    input = input.replace(",", " ")
    return youtube.run(input)

In [ ]:
# Define tools
tools = [
    Tool.from_function(
        name="Movie Chat",
        description="For when you need to chat about movies. The question will be a string. Return a string.",
        func=chat_chain.invoke,
    ),
    Tool.from_function(
        name="Movie Trailer Search",
        description="Use when needing to find a movie trailer. The question will include the word trailer. Return a link to a YouTube video.",
        func=call_trailer_search,
    ),
    Tool.from_function(
        name="Movie Plot Search",
        description="For when you need to compare a plot to a movie. The question will be a string. Return a string.",
        func=retriever.invoke,
    ),
]

print(tools)

In [ ]:
# Create the agent and agent executor
agent_prompt = hub.pull("hwchase17/react-chat")
agent = create_react_agent(llm, tools, agent_prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, handle_parsing_errors=True)

# Create the chat agent with message history
chat_agent = RunnableWithMessageHistory(
    agent_executor,
    get_memory,
    input_messages_key="input",
    history_messages_key="chat_history"
)

print(chat_agent)

In [ ]:
response = chat_agent.invoke(
    {"input": "retrieve info using this: <id>: 4:cf7c648b-d8fa-4477-8e38-244d460c8163:137"},
    {"configurable": {"session_id": SESSION_ID}}
)

print(response.get("output"))